# MLTagger

## Model
Going by the name model.py, this script contains code for MLTagger model where layers are constructed in `construct_network`. `build_vocabs`, `initialize_sesison` and `preload_word_embeddings` are supporting functions for data preprocessing (e.g. usage of word embeddings - GLoVe). 

## Available functions in model.py

```Python
class MLTModel(object):
    self.config = config
    self.UNK = "<unk>"
    self.CUNK = "<cunk>"
    self.word2id = None
    self.char2id = None
    self.singletons = None

    def build_vocabs(self, data_train, data_dev, data_test, embedding_path=None):
        self.word2id
        self.char2id
        self.singletons
        
    def construct_network(self)
    
    def construct_lmcost(self, input_tensor_fw, input_tensor_bw, sentence_lengths, target_ids, lmcost_type, name):
        return cost
        
    def _construct_lmcost(self, input_tensor, lmcost_max_vocab_size, lmcost_mask, target_ids, name):
        return tf.reduce_sum(lmcost_loss)
    
    def construct_optimizer(self, opt_strategy, loss, learningrate, clip):
        return train_op
    
    def preload_word_embeddings(self, embedding_path)
    
    def translate2id(self, token, token2id, unk_token, lowercase=False, replace_digits=False, singletons=None, singletons_prob=0.0):
        return token_id
    
    def create_input_dictionary_for_batch(self, batch, is_training, learningrate):
        return input_dictionary
    
    def process_batch(self, batch, is_training, learningrate):
        return cost, sentence_scores, token_scores
    
    def initialize_session(self)
    
    def get_parameter_count(self):
        return total_parameters
    
    def get_parameter_count_without_word_embeddings(self):
        return self.get_parameter_count() - variable_parameters
    
    def save(self, filename)
    
    def load(filename, new_config=None)
    
    def load_params(self, filename)
```

## Import Files

In [ ]:
import collections
import tensorflow as tf
import rebui
import numpy
from tensorflow.python.framework import ops
from tensorflow.python.ops import math_ops

try:
    import cPickle as pickle
except:
    import pickle

### build_vocabs
This function takes in training, dev and test dataset and:
1. Perform a frequency count of character and words 
2. Populate word2id, char2id starting with the most popular word and char
3. Generate singletons

Default:
- `vocab_include_devtest` = `True`
- `lowercase` = `True`
- `replace_digits` = `True`
- `vocab_only_embedded` = `False`

char_counter:
```Python
Counter({'e': 195183, 't': 152545, 'o': 137220, 'a': 127797, 'n': 108966, 'i': 102277, 's': 95198, 'r': 88551, 'h': 81427, 'l': 68566, 'd': 56608, 'u': 51079, 'y': 43276, 'm': 41594, 'c': 40632, 'w': 40504, 'f': 31992, 'g': 30874, 'p': 29857, '.': 24838, 'I': 24032, 'b': 23455, 'v': 19913, ',': 17613, 'k': 16165, 'T': 6985, "'": 6309, 'A': 5471, 'S': 4340, 'E': 3820, 'O': 3264, 'D': 3025, 'x': 2813, 'H': 2758, 'M': 2751, 'W': 2700, 'F': 2510, 'N': 2428, 'R': 2335, 'L': 2287, 'C': 2215, 'B': 2205, 'Y': 2120, 'j': 2096, '0': 1987, 'P': 1701, '"': 1548, '1': 1430, 'U': 1281, '!': 1200, '?': 1173, 'J': 1152, '-': 1068, 'q': 988, ':': 988, 'z': 878, 'G': 806, '2': 743, 'K': 631, '5': 551, '3': 520, 'V': 513, '9': 415, '4': 305, ')': 287, '(': 286, ';': 202, '/': 163, '7': 155, '6': 111, '8': 90, '£': 83, 'X': 70, 'Z':29, 'Q': 27, '%': 15, '&': 14, 'ö': 7, 'ô': 7, 'ã': 6, 'ë': 5, 'é': 5, '=': 4, '$': 3, '*': 3, 'ü': 2, 'à': 2, 'á': 2, 'ê': 1, '`': 1, 'ñ': 1, 'ä': 1, '+': 1})
```

word_counter:
```Python
Counter({'.': 22909, 'i': 19532, 'the': 17909, ',': 17574, 'to': 15510, 'and': 9925, 'a': 7098, 'in': 7090, 'you': 6707, 'of': 6495, 'it': 6435, 'was': 5724, 'that': 5329, 'for': 4465, 'is': 4432, 'my': 4279, 'have': 4112, 'we': 3490, 'at': 2967, 'be': 2951, 'would': 2855, 'but': 2742, 'because': 2645, "n't": 2619, 'me': 2605, 'like': 2560, 'your': 2479, 'very': 2429, 'not': 2315, 'with': 2232, 'this': 2227, 'are': 2166, 'about': 2142, 'as': 2121, 'on': 2091, 'will': 2030, 'all': 2025, 'do': 2005, 'had': 1920, 'am': 1891, 'show': 1881, 'so': 1881, 'can': 1717, 'there': 1646, 'they': 1616, ....})
```

self.char2id:
```Python
OrderedDict([('<cunk>', 0), ('e', 1), ('t', 2), ('o', 3), ('a', 4), ('n', 5), ('i', 6), ('s', 7), ('r', 8), ('h', 9), ('l', 10), ('d', 11), ('u', 12), ('y', 13), ('m', 14), ('c', 15), ('w', 16), ('f', 17), ('g', 18), ('p', 19), ('.', 20), ('I', 21), ('b', 22), ('v', 23), (',', 24), ('k', 25), ('T', 26), ("'", 27), ('A', 28), ('S', 29), ('E', 30), ('O', 31), ('D', 32), ('x', 33), ('H', 34), ('M', 35), ('W', 36), ('F', 37), ('N', 38), ('R', 39), ('L', 40), ('C', 41), ('B', 42), ('Y', 43), ('j', 44), ('0', 45), ('P', 46), ('"', 47), ('1', 48), ('U', 49), ('!', 50), ('?', 51), ('J', 52), ('-', 53), ('q', 54), (':', 55), ('z', 56), ('G', 57), ('2', 58), ('K', 59), ('5', 60), ('3', 61), ('V', 62), ('9', 63), ('4', 64), (')', 65), ('(', 66), (';', 67), ('/', 68), ('7', 69), ('6', 70), ('8', 71), ('£', 72), ('X', 73), ('Z', 74), ('Q', 75), ('%', 76), ('&', 77), ('ö', 78), ('ô', 79), ('ã', 80), ('ë', 81), ('é', 82), ('=', 83), ('$', 84), ('*', 85), ('ü', 86), ('à', 87), ('á', 88), ('ê', 89), ('`', 90), ('ñ', 91), ('ä', 92), ('+', 93)])
```

self.word2id
```Python
OrderedDict([('<unk>', 0), ('.', 1), ('i', 2), ('the', 3), (',', 4), ('to', 5), ('and', 6), ('a', 7), ('in', 8), ('you', 9), ('of', 10), ('it', 11), ('was', 12), ('that', 13), ('for', 14), ('is', 15), ('my', 16), ('have', 17), ('we', 18), ('at', 19), ('be', 20), ('would', 21), ('but', 22), ('because', 23), ("n't", 24), ('me', 25), ('like', 26), ('your', 27), ('very', 28), ('not', 29), ('with', 30), ('this', 31), ('are', 32), ('about', 33), ('as', 34), ('on', 35), ('will', 36), ('all', 37),('do', 38), ('had', 39), ('am', 40), ('show', 41), ('so', 42), ('can', 43), ('there', 44), ('they', 45), ('from', 46), ('"', 47), ('if', 48), ('time', 49), ('when',50), ('people', 51), ('some', 52), ('could', 53), ("'s", 54), ('were', 55), ('our', 56), ('dear', 57), ('!', 58), ('think', 59), ('good', 60), ('or', 61), ('what', 62), ('?', 63), ('more', 64), ('money', 65), ('go', 66), ('which', 67), ('one', 68), ('know', 69), ('shopping', 70), ('life', 71), ('also', 72), ('how', 73), ('first', 74), ('only', 75), ("'m", 76), ('he', 77), ('she', 78), ('yours', 79), ('really', 80), ('clothes', 81), ('them', 82), ('theatre', 83), ('much', 84), ('an', 85), ('see', 86), ('want', 87), ('should', 88), ('by', 89), ('after', 90), ('school', 91), ('other', 92), ('her', 93), ('been', 94), ('pat', 95), ('did', 96), ('letter', 97), ('take', 98), ('us', 99), ('concert', 100), ('forward', 101), ('going', 102), ('advertisement', 103), ('last', 104), ... ])
```

self.singletons
```Python
{'disorganized', 'unpredictable', 'beans', 'andersen', 'liyana', 'sequences', 'dissappeared', 'invites', 'greeks', 'knowledges', 'airborn', 'ezier', 'vacancies', 'charachter', 'iii', 'matos', 'ambisious', 'torres', 'professions', 'hansom', 'sentance', 'alter', 'undstand', 'minority', 'entertainers', 'biscount', 'dure', 'knowleage', 'learn.', 'growns', 'enjoyied', 'quitely', 'behaviers', 'troubel', 'equippments', 'rebuild', 'sining', 'enhance', 'besorrted', 're-arrange', 'whed', 'vargas', 'sleeper', 'undenyable', 'slipping', 'desappointing', 'inhabitants', 'ill-fitted', 'religion', 'dish-washer', 'mile-long', 'u.s', 'inspite', 'tempting', 'commanded', 'evolutive', 'enjoin', 'kraus', 'breathed', 'sequence', 'strenge', 'majesty', 'suit.', 'precarious', 'safina', 'v', 'wage', 'parade', 'abbsend', 'fete', 'thai', 'minimise', 'remote', 'thatt', 'cleanest', 'trusting', 'selfishly', 'aftering', 'adimnistration', 'exception', 'overdrawn','lode', 'absuletely', 'accutely', 'beginers', 'repeard', 'discoteks', 'killer', 'scholar', 'unfourtunately', 'els', 'marked', 'roddos', 'consume', 'vains', 'catchs', 'retrieve', 'torture', 'amasing', 'comprehensible', 'mea', 'retelling', 'talent.', 'noice', 'poisonous', 'illness..', 'hangry', 'shoplifters', 'america.', 'imprisonment', 'gritted', 'tikets', 'sathurday', 'steam', 'satellite', 'recollet', 'heay', 'areva', '?look', 'friendness', 'dragon', 'voices', 'tired.', 'appriciation', 'richey', 'triangular', 'partial', 'patners', 'motivations', 'foreginers', 'dann', 'dissision', 'firstable', 'techer', 'definity', 'despleased', 'appropiated', 'doomed', 'kozin', 'mühlenstrasse', 'welcoming', 'hooting', 'smellt', 'lavaux', 'sientific', 'medecine', 'previst', 'fiance', 'verybody', 'netserver', 'mo', 'advicing', 'recone', 'buffett', 'abondone', 'owe', 'interviewer', 'honorable', 'gymnasium', 'setisfaid', 'seill', 'adittion', 'quantities', 'handled', 'wrought', 'unforgivly', 'explicitly', 'emsamblish', 'tanaka', 'desadventages', 'robbie', 'assistent', 'joachim', 'sweater', 'regretting', 'dinara', 'tutry', 'durty', 'dekoreting', 'grabed', 'heyman', 'unaware', 'entertaints', 'proformed', 'african', 'day.', 'immigration', 'whot', 'fobia', 'intesting', 'placement', 'guards', 'focalized', 'immidiate', 'beek', 'brocher', 'maraya', 'sportif', 'detailed', 'sevral', 'santigo', 'experiment', 'embarased', 'marrige', 'gardent', 'thumbing', 'slope', 'calendar', 'schulsson', 'karin', 'high-pitch', 'repairement', 'hase', 'anis', 'plesent', 'crowding', ...}
```

In [5]:
%%script false
def build_vocabs(self, data_train, data_dev, data_test, embedding_path=None):
    # Constructing data_source so that vocab includes dev and test sets
    # vocab_include_devtest - Load words from dev and test sets also into the vocabulary. 
    # If they don't appear in the training set, they will have the default representations from the preloaded embeddings.
    # vocab_include_devtest = True
    data_source = list(data_train)
    if self.config["vocab_include_devtest"]:
        if data_dev != None:
            data_source += data_dev
        if data_test != None:
            data_source += data_test

    # Build char_counter
    char_counter = collections.Counter()
    for sentence in data_source:
        for word in sentence:
            # word = ['concerning', 'c']
            char_counter.update(word[0])

    # Populate self.char2id
    self.char2id = collections.OrderedDict([(self.CUNK, 0)])
    for char, count in char_counter.most_common():
        if char not in self.char2id:
            self.char2id[char] = len(self.char2id)

    # Build word_counter
    word_counter = collections.Counter()
    for sentence in data_source:
        for word in sentence:
            w = word[0]
            if self.config["lowercase"] == True:
                w = w.lower()
            if self.config["replace_digits"] == True:
                w = re.sub(r'\d', '0', w)
            word_counter[w] += 1

    # Populate self.word2id
    self.word2id = collections.OrderedDict([(self.UNK, 0)])
    for word, count in word_counter.most_common():
        if self.config["min_word_freq"] <= 0 or count >= self.config["min_word_freq"]:
            if word not in self.word2id:
                self.word2id[word] = len(self.word2id)

    # Build Singletons (words that only appear once)
    self.singletons = set([word for word in word_counter if word_counter[word] == 1])

    # If embedding_path (which is path for preload vector e.g. GloVe) is configured 
    # and if vocab_only_embedded is True, word2id will be constructed with word that 
    # only exists in embeddings, all other words not found will be excluded.
    if embedding_path != None and self.config["vocab_only_embedded"] == True:
        self.embedding_vocab = set([self.UNK])
        with open(embedding_path, 'r') as f:
            for line in f:
                line_parts = line.strip().split()
                if len(line_parts) <= 2:
                    continue
                w = line_parts[0]
                if self.config["lowercase"] == True:
                    w = w.lower()
                if self.config["replace_digits"] == True:
                    w = re.sub(r'\d', '0', w)
                self.embedding_vocab.add(w)
        word2id_revised = collections.OrderedDict()
        for word in self.word2id:
            # If the word appears in the corpus but not in embeddings, do not add it into the new word2id
            if word in embedding_vocab and word not in word2id_revised:
                word2id_revised[word] = len(word2id_revised)
        self.word2id = word2id_revised

    print("n_words: " + str(len(self.word2id)))
    print("n_chars: " + str(len(self.char2id)))
    print("n_singletons: " + str(len(self.singletons)))


In [ ]:
class MLTModel(object):
    def __init__(self, config):
        self.config = config

        self.UNK = "<unk>"
        self.CUNK = "<cunk>"

        self.word2id = None
        self.char2id = None
        self.singletons = None

    def build_vocabs(self, data_train, data_dev, data_test, embedding_path=None):
        # Constructing data_source so that vocab includes dev and test sets
        # vocab_include_devtest - Load words from dev and test sets also into the vocabulary. 
        # If they don't appear in the training set, they will have the default representations from the preloaded embeddings.
        # vocab_include_devtest = True
        data_source = list(data_train)
        if self.config["vocab_include_devtest"]:
            if data_dev != None:
                data_source += data_dev
            if data_test != None:
                data_source += data_test
        
        # Build char_counter
        char_counter = collections.Counter()
        for sentence in data_source:
            for word in sentence:
                # word = ['concerning', 'c']
                char_counter.update(word[0])
        
        # Populate self.char2id
        self.char2id = collections.OrderedDict([(self.CUNK, 0)])
        for char, count in char_counter.most_common():
            if char not in self.char2id:
                self.char2id[char] = len(self.char2id)
        
        # Build word_counter
        word_counter = collections.Counter()
        for sentence in data_source:
            for word in sentence:
                w = word[0]
                if self.config["lowercase"] == True:
                    w = w.lower()
                if self.config["replace_digits"] == True:
                    w = re.sub(r'\d', '0', w)
                word_counter[w] += 1

        # Populate self.word2id
        self.word2id = collections.OrderedDict([(self.UNK, 0)])
        for word, count in word_counter.most_common():
            if self.config["min_word_freq"] <= 0 or count >= self.config["min_word_freq"]:
                if word not in self.word2id:
                    self.word2id[word] = len(self.word2id)
    
        # Build Singletons (words that only appear once)
        self.singletons = set([word for word in word_counter if word_counter[word] == 1])

        # If embedding_path (which is path for preload vector e.g. GloVe) is configured 
        # and if vocab_only_embedded is True, word2id will be constructed with word that 
        # only exists in embeddings, all other words not found will be excluded.
        if embedding_path != None and self.config["vocab_only_embedded"] == True:
            self.embedding_vocab = set([self.UNK])
            with open(embedding_path, 'r') as f:
                for line in f:
                    line_parts = line.strip().split()
                    if len(line_parts) <= 2:
                        continue
                    w = line_parts[0]
                    if self.config["lowercase"] == True:
                        w = w.lower()
                    if self.config["replace_digits"] == True:
                        w = re.sub(r'\d', '0', w)
                    self.embedding_vocab.add(w)
            word2id_revised = collections.OrderedDict()
            for word in self.word2id:
                # If the word appears in the corpus but not in embeddings, do not add it into the new word2id
                if word in embedding_vocab and word not in word2id_revised:
                    word2id_revised[word] = len(word2id_revised)
            self.word2id = word2id_revised

        print("n_words: " + str(len(self.word2id)))
        print("n_chars: " + str(len(self.char2id)))
        print("n_singletons: " + str(len(self.singletons)))


    def construct_network(self):
        self.word_ids = tf.placeholder(tf.int32, [None, None], name="word_ids")
        self.char_ids = tf.placeholder(tf.int32, [None, None, None], name="char_ids")
        self.sentence_lengths = tf.placeholder(tf.int32, [None], name="sentence_lengths")
        self.word_lengths = tf.placeholder(tf.int32, [None, None], name="word_lengths")
        self.sentence_labels = tf.placeholder(tf.float32, [None,], name="sentence_labels")
        self.word_labels = tf.placeholder(tf.float32, [None,None], name="word_labels")
        self.word_objective_weights = tf.placeholder(tf.float32, [None,None], name="word_objective_weights")
        self.sentence_objective_weights = tf.placeholder(tf.float32, [None], name="sentence_objective_weights")
        self.learningrate = tf.placeholder(tf.float32, name="learningrate")
        self.is_training = tf.placeholder(tf.int32, name="is_training")

        self.loss = 0.0
        input_tensor = None
        input_vector_size = 0

        self.initializer = None
        if self.config["initializer"] == "normal":
            self.initializer = tf.random_normal_initializer(mean=0.0, stddev=0.1)
        elif self.config["initializer"] == "glorot":
            self.initializer = tf.glorot_uniform_initializer()
        elif self.config["initializer"] == "xavier":
            self.initializer = tf.glorot_normal_initializer()

        zeros_initializer = tf.zeros_initializer()

        self.word_embeddings = tf.get_variable("word_embeddings", 
            shape=[len(self.word2id), self.config["word_embedding_size"]], 
            initializer=(zeros_initializer if self.config["emb_initial_zero"] == True else self.initializer),
            trainable=(True if self.config["train_embeddings"] == True else False))

        # word_embeddings [13470 300][[0.0164626278 -0.0187288448 -0.0165187102 -0.00846466795 -0.000238586217]...]
        # self.word_embeddings = tf.Print(self.word_embeddings, [tf.shape(self.word_embeddings), self.word_embeddings], 'word_embeddings ', summarize=5)

        input_tensor = tf.nn.embedding_lookup(self.word_embeddings, self.word_ids)

        # input_tensor [32 42 300][[[0.033284 -0.040754 -0.048377 0.12017 -0.13915]]...]
        # input_tensor = tf.Print(input_tensor, [tf.shape(input_tensor), input_tensor], 'input_tensor ', summarize=5)

        # word_embedding_size = 300        
        input_vector_size = self.config["word_embedding_size"]

        # char_embedding_size = 100, char_recurrent_size = 100
        if self.config["char_embedding_size"] > 0 and self.config["char_recurrent_size"] > 0:
            with tf.variable_scope("chars"), tf.control_dependencies([tf.assert_equal(tf.shape(self.char_ids)[2], tf.reduce_max(self.word_lengths), message="Char dimensions don't match")]):
                self.char_embeddings = tf.get_variable("char_embeddings", 
                    shape=[len(self.char2id), self.config["char_embedding_size"]], 
                    initializer=self.initializer, 
                    trainable=True)

                # self.char_embeddings [97 100][[-0.0284440666 -0.0544790775 -0.0190391093 0.170494288 0.133321077]...]
                # self.char_embeddings = tf.Print(self.char_embeddings, [tf.shape(self.char_embeddings), self.char_embeddings], 'self.char_embeddings ', summarize=5)

                # char_input_tensor returns tensor containing embeddings of respective char_ids
                char_input_tensor = tf.nn.embedding_lookup(self.char_embeddings, self.char_ids)
                s = tf.shape(char_input_tensor)

                # char_input_tensor [32 42 14 100][[[[-0.133647785 -0.0104248524 -0.0767552108 0.157381654 -0.0219091922]]]...]
                # char_input_tensor = tf.Print(char_input_tensor, [tf.shape(char_input_tensor), char_input_tensor], 'char_input_tensor ', summarize=5)

                # Reshaping char_input_tensor and word_lengths - Why?
                # self.char_embeddings (97, 100), word_lengths (32, 42)
                # char_input_tensor.shape (1344, 14, 100)
                char_input_tensor = tf.reshape(char_input_tensor, shape=[s[0]*s[1], s[2], self.config["char_embedding_size"]])

                # char_input_tensor [1344 14 100][[[-0.133647785 -0.0104248524 -0.0767552108 0.157381654 -0.0219091922]]...]
                # char_input_tensor = tf.Print(char_input_tensor, [tf.shape(char_input_tensor), char_input_tensor], 'char_input_tensor ', summarize=5)

                _word_lengths = tf.reshape(self.word_lengths, shape=[s[0]*s[1]])

                # lstm_use_peepholes = False
                char_lstm_cell_fw = tf.nn.rnn_cell.LSTMCell(self.config["char_recurrent_size"], 
                    use_peepholes=self.config["lstm_use_peepholes"], 
                    state_is_tuple=True, 
                    initializer=self.initializer,
                    reuse=False)
                char_lstm_cell_bw = tf.nn.rnn_cell.LSTMCell(self.config["char_recurrent_size"], 
                    use_peepholes=self.config["lstm_use_peepholes"], 
                    state_is_tuple=True, 
                    initializer=self.initializer,
                    reuse=False)

                char_lstm_outputs = tf.nn.bidirectional_dynamic_rnn(
                    char_lstm_cell_fw, char_lstm_cell_bw, 
                    char_input_tensor, sequence_length=_word_lengths, 
                    dtype=tf.float32, time_major=False)
                _, ((_, char_output_fw), (_, char_output_bw)) = char_lstm_outputs

                # char_output_fw [1344 100][[-0.0202908032 0.0152226407 0.0122073945 -0.000989505905 -0.0187146086]...]
                # char_output_fw = tf.Print(char_output_fw, [tf.shape(char_output_fw), char_output_fw], 'char_output_fw ', summarize=5)

                # char_output_bw [1344 100][[0.0369782448 0.0142420894 0.0041682804 0.0185243599 -0.0126576656]...]
                # char_output_bw = tf.Print(char_output_bw, [tf.shape(char_output_bw), char_output_bw], 'char_output_bw ', summarize=5)

                char_output_tensor = tf.concat([char_output_fw, char_output_bw], axis=-1)

                # char_output_tensor [1344 200][[0.00968829822 0.010671746 0.00669185 0.0245511737 0.00249310443]...]
                # char_output_tensor = tf.Print(char_output_tensor, [tf.shape(char_output_tensor), char_output_tensor], 'char_output_tensor ', summarize=5)

                char_output_tensor = tf.reshape(char_output_tensor, shape=[s[0], s[1], 2 * self.config["char_recurrent_size"]])

                # char_output_tensor [32 42 200][[[-0.012360299 -0.00901429448 0.0371902511 -0.0109808315 0.00917478558]]...]
                # char_output_tensor = tf.Print(char_output_tensor, [tf.shape(char_output_tensor), char_output_tensor], 'char_output_tensor ', summarize=5)

                char_output_vector_size = 2 * self.config["char_recurrent_size"]

                # lmcost_char_gamma = 0.0
                if self.config["lmcost_char_gamma"] > 0.0:
                    self.loss += self.config["lmcost_char_gamma"] * self.construct_lmcost(char_output_tensor, char_output_tensor, self.sentence_lengths, self.word_ids, "separate", "lmcost_char_separate")
                
                # lmcost_joint_char_gamma = 0.0
                if self.config["lmcost_joint_char_gamma"] > 0.0:
                    self.loss += self.config["lmcost_joint_char_gamma"] * self.construct_lmcost(char_output_tensor, char_output_tensor, self.sentence_lengths, self.word_ids, "joint", "lmcost_char_joint")
                
                # char_hidden_layer_size =50
                if self.config["char_hidden_layer_size"] > 0:
                    char_output_tensor = tf.layers.dense(char_output_tensor, self.config["char_hidden_layer_size"], activation=tf.tanh, kernel_initializer=self.initializer)
                    char_output_vector_size = self.config["char_hidden_layer_size"]

                    # char_output_tensor [32 42 50][[[0.0127831129 -0.00378674385 -0.0283287913 -0.021209931 0.000687278458]]...]
                    # char_output_tensor = tf.Print(char_output_tensor, [tf.shape(char_output_tensor), char_output_tensor], 'char_output_tensor ', summarize=5)

                # char_integration_method = concat
                if self.config["char_integration_method"] == "concat":
                    input_tensor = tf.concat([input_tensor, char_output_tensor], axis=-1)

                    # input_tensor [32 42 350][[[0.033284 -0.040754 -0.048377 0.12017 -0.13915]]...]
                    # input_tensor = tf.Print(input_tensor, [tf.shape(input_tensor), input_tensor], 'input_tensor ', summarize=5)

                    input_vector_size += char_output_vector_size
                elif self.config["char_integration_method"] == "none":
                    input_tensor = input_tensor
                else:
                    raise ValueError("Unknown char integration method")

        # This is after concatenation of word_representations
        self.word_representations = input_tensor

        # dropout_input = 0.5
        dropout_input = self.config["dropout_input"] * tf.cast(self.is_training, tf.float32) + (1.0 - tf.cast(self.is_training, tf.float32))
        input_tensor =  tf.nn.dropout(input_tensor, dropout_input, name="dropout_word")

        # input_tensor [32 42 350][[[0 -0 -0 0.24034 -0.2783]]...]
        # input_tensor = tf.Print(input_tensor, [tf.shape(input_tensor), input_tensor], 'input_tensor ', summarize=5)

        # word_recurrent_size = 300
        word_lstm_cell_fw = tf.nn.rnn_cell.LSTMCell(self.config["word_recurrent_size"], 
            use_peepholes=self.config["lstm_use_peepholes"], 
            state_is_tuple=True, 
            initializer=self.initializer,
            reuse=False)
        word_lstm_cell_bw = tf.nn.rnn_cell.LSTMCell(self.config["word_recurrent_size"], 
            use_peepholes=self.config["lstm_use_peepholes"], 
            state_is_tuple=True, 
            initializer=self.initializer,
            reuse=False)

        with tf.control_dependencies([tf.assert_equal(tf.shape(self.word_ids)[1], tf.reduce_max(self.sentence_lengths), message="Sentence dimensions don't match")]):
            (lstm_outputs_fw, lstm_outputs_bw), ((_, lstm_output_fw), (_, lstm_output_bw)) = tf.nn.bidirectional_dynamic_rnn(word_lstm_cell_fw, word_lstm_cell_bw, input_tensor, sequence_length=self.sentence_lengths, dtype=tf.float32, time_major=False)

        # dropout_word_lstm = 0.5
        dropout_word_lstm = self.config["dropout_word_lstm"] * tf.cast(self.is_training, tf.float32) + (1.0 - tf.cast(self.is_training, tf.float32))
        # noise_shape?
        # word_recurrent_size = 300
        lstm_outputs_fw =  tf.nn.dropout(lstm_outputs_fw, dropout_word_lstm, noise_shape=tf.convert_to_tensor([tf.shape(self.word_ids)[0],1,self.config["word_recurrent_size"]], dtype=tf.int32))
        lstm_outputs_bw =  tf.nn.dropout(lstm_outputs_bw, dropout_word_lstm, noise_shape=tf.convert_to_tensor([tf.shape(self.word_ids)[0],1,self.config["word_recurrent_size"]], dtype=tf.int32))

        # lstm_outputs_fw [32 42 300][[[0.27246663 0 0.0123384269 -0.156472668 0.175036415]]...]
        # lstm_outputs_fw = tf.Print(lstm_outputs_fw, [tf.shape(lstm_outputs_fw), lstm_outputs_fw], 'lstm_outputs_fw ', summarize=5)

        # lstm_outputs_bw [32 42 300][[[-0 -0.0373351909 0 0 0.186578169]]...]
        # lstm_outputs_bw = tf.Print(lstm_outputs_bw, [tf.shape(lstm_outputs_bw), lstm_outputs_bw], 'lstm_outputs_bw ', summarize=5)

        lstm_outputs = tf.concat([lstm_outputs_fw, lstm_outputs_bw], -1)

        # lstm_outputs [32 42 600][[[0.277638972 -0.00427084789 0 0.00701316865 -0]]...]
        # lstm_outputs = tf.Print(lstm_outputs, [tf.shape(lstm_outputs), lstm_outputs], 'lstm_outputs ', summarize=5)

        # whidden_layer_size = 200
        if self.config["whidden_layer_size"] > 0:
            lstm_outputs = tf.layers.dense(lstm_outputs, self.config["whidden_layer_size"], activation=tf.tanh, kernel_initializer=self.initializer)

            # lstm_outputs [32 42 200][[[-0.0930339 -0.07592123 -0.400819033 0.147020906 -0.0770602524]]...]
            # lstm_outputs = tf.Print(lstm_outputs, [tf.shape(lstm_outputs), lstm_outputs], 'lstm_outputs ', summarize=5)

        self.lstm_outputs = lstm_outputs

        lstm_output = tf.concat([lstm_output_fw, lstm_output_bw], -1)
        lstm_output = tf.nn.dropout(lstm_output, dropout_word_lstm)

        # sentence_composition = attention
        if self.config["sentence_composition"] == "last":
            processed_tensor = lstm_output
            self.attention_weights_unnormalised = tf.zeros_like(self.word_ids, dtype=tf.float32)
        elif self.config["sentence_composition"] == "attention":
            with tf.variable_scope("attention"):
                attention_evidence = tf.layers.dense(lstm_outputs, self.config["attention_evidence_size"], activation=tf.tanh, kernel_initializer=self.initializer)

                # attention_evidence [32 42 100][[[-0.1122666 -0.208284497 0.0635408163 0.072117269 0.114323549]]...]
                # attention_evidence = tf.Print(attention_evidence, [tf.shape(attention_evidence), attention_evidence], 'attention_evidence ', summarize=5)

                attention_weights = tf.layers.dense(attention_evidence, 1, activation=None, kernel_initializer=self.initializer)

                # attention_weights [32 42 1][[[0.175914466][0.114666343][0.0859841406][0.220286131][0.345916331]]...]
                # attention_weights = tf.Print(attention_weights, [tf.shape(attention_weights), attention_weights], 'attention_weights ', summarize=5)

                attention_weights = tf.reshape(attention_weights, shape=tf.shape(self.word_ids))

                # attention_weights [32 42][[0.257553369 0.128167823 0.15162158 0.0321515091 0.0484975688]...]
                # attention_weights = tf.Print(attention_weights, [tf.shape(attention_weights), attention_weights], 'attention_weights ', summarize=5)

                # attention_activation = soft
                if self.config["attention_activation"] == "sharp":
                    attention_weights = tf.exp(attention_weights)
                elif self.config["attention_activation"] == "soft":
                    attention_weights = tf.sigmoid(attention_weights)
                elif self.config["attention_activation"] == "linear":
                    pass
                else:
                    raise ValueError("Unknown activation for attention: " + str(self.config["attention_activation"]))

                word_objective_loss = tf.square(attention_weights - self.word_labels)
                word_objective_loss = tf.where(tf.sequence_mask(self.sentence_lengths), word_objective_loss, tf.zeros_like(word_objective_loss))
                self.loss += self.config["word_objective_weight"] * tf.reduce_sum(self.word_objective_weights * word_objective_loss)

                self.attention_weights_unnormalised = attention_weights
                attention_weights = tf.where(tf.sequence_mask(self.sentence_lengths), attention_weights, tf.zeros_like(attention_weights))
                attention_weights = attention_weights / tf.reduce_sum(attention_weights, 1, keep_dims=True)
                processed_tensor = tf.reduce_sum(lstm_outputs * attention_weights[:,:,numpy.newaxis], 1)

                # processed_tensor [32 200][[0.0998472124 -0.15800871 0.180494159 0.212085128 -0.0436996967]...]
                # processed_tensor = tf.Print(processed_tensor, [tf.shape(processed_tensor), processed_tensor], 'processed_tensor ', summarize=5)

        # hidden_layer_size = 50
        if self.config["hidden_layer_size"] > 0:
            processed_tensor = tf.layers.dense(processed_tensor, self.config["hidden_layer_size"], activation=tf.tanh, kernel_initializer=self.initializer)

            # processed_tensor [32 50][[0.0976842418 0.178458333 -0.119485192 0.132142678 0.169890434]...]
            # processed_tensor = tf.Print(processed_tensor, [tf.shape(processed_tensor), processed_tensor], 'processed_tensor ', summarize=5)

        self.sentence_scores = tf.layers.dense(processed_tensor, 1, activation=tf.sigmoid, kernel_initializer=self.initializer, name="output_ff")

        # self.sentence_scores (after dense) [32 1][[0.481682301][0.566274941][0.535907149][0.626509309][0.585899055]...]
        # self.sentence_scores = tf.Print(self.sentence_scores, [tf.shape(self.sentence_scores), self.sentence_scores], 'self.sentence_scores (after dense) ', summarize=5)

        self.sentence_scores = tf.reshape(self.sentence_scores, shape=[tf.shape(processed_tensor)[0]])

        # self.sentence_scores (after reshape) [32][0.515106499 0.51553309 0.406303078 0.582218766 0.401411533...]
        # self.sentence_scores = tf.Print(self.sentence_scores, [tf.shape(self.sentence_scores), self.sentence_scores], 'self.sentence_scores (after reshape) ', summarize=5)

        self.loss += self.config["sentence_objective_weight"] * tf.reduce_sum(self.sentence_objective_weights * tf.square(self.sentence_scores - self.sentence_labels))

        # attention_objective_weight = 0.01
        if self.config["attention_objective_weight"] > 0.0:
            self.loss += self.config["attention_objective_weight"] * \
                (tf.reduce_sum(
                    self.sentence_objective_weights *tf.square(
                        tf.reduce_max(
                            tf.where(
                                tf.sequence_mask(self.sentence_lengths), 
                                self.attention_weights_unnormalised, 
                                tf.zeros_like(self.attention_weights_unnormalised) - 1e6), 
                            axis=-1) - self.sentence_labels))
                + 
                tf.reduce_sum(
                    self.sentence_objective_weights * tf.square(
                        tf.reduce_min(
                            tf.where(
                                tf.sequence_mask(self.sentence_lengths), 
                                self.attention_weights_unnormalised, 
                                tf.zeros_like(self.attention_weights_unnormalised) + 1e6), 
                            axis=-1) - 0.0)))

        self.token_scores = [tf.where(tf.sequence_mask(self.sentence_lengths), self.attention_weights_unnormalised, tf.zeros_like(self.attention_weights_unnormalised) - 1e6)]

        if self.config["lmcost_lstm_gamma"] > 0.0:
            self.loss += self.config["lmcost_lstm_gamma"] * self.construct_lmcost(lstm_outputs_fw, lstm_outputs_bw, self.sentence_lengths, self.word_ids, "separate", "lmcost_lstm_separate")
        if self.config["lmcost_joint_lstm_gamma"] > 0.0:
            self.loss += self.config["lmcost_joint_lstm_gamma"] * self.construct_lmcost(lstm_outputs_fw, lstm_outputs_bw, self.sentence_lengths, self.word_ids, "joint", "lmcost_lstm_joint")

        self.train_op = self.construct_optimizer(self.config["opt_strategy"], self.loss, self.learningrate, self.config["clip"])

    def construct_lmcost(self, input_tensor_fw, input_tensor_bw, sentence_lengths, target_ids, lmcost_type, name):
        with tf.variable_scope(name):
            lmcost_max_vocab_size = min(len(self.word2id), self.config["lmcost_max_vocab_size"])
            target_ids = tf.where(tf.greater_equal(target_ids, lmcost_max_vocab_size-1), x=(lmcost_max_vocab_size-1)+tf.zeros_like(target_ids), y=target_ids)
            cost = 0.0
            if lmcost_type == "separate":
                lmcost_fw_mask = tf.sequence_mask(sentence_lengths, maxlen=tf.shape(target_ids)[1])[:,1:]
                lmcost_bw_mask = tf.sequence_mask(sentence_lengths, maxlen=tf.shape(target_ids)[1])[:,:-1]
                lmcost_fw = self._construct_lmcost(input_tensor_fw[:,:-1,:], lmcost_max_vocab_size, lmcost_fw_mask, target_ids[:,1:], name=name+"_fw")
                lmcost_bw = self._construct_lmcost(input_tensor_bw[:,1:,:], lmcost_max_vocab_size, lmcost_bw_mask, target_ids[:,:-1], name=name+"_bw")
                cost += lmcost_fw + lmcost_bw
            elif lmcost_type == "joint":
                joint_input_tensor = tf.concat([input_tensor_fw[:,:-2,:], input_tensor_bw[:,2:,:]], axis=-1)
                lmcost_mask = tf.sequence_mask(sentence_lengths, maxlen=tf.shape(target_ids)[1])[:,1:-1]
                cost += self._construct_lmcost(joint_input_tensor, lmcost_max_vocab_size, lmcost_mask, target_ids[:,1:-1], name=name+"_joint")
            else:
                raise ValueError("Unknown lmcost_type: " + str(lmcost_type))
            return cost


    def _construct_lmcost(self, input_tensor, lmcost_max_vocab_size, lmcost_mask, target_ids, name):
        with tf.variable_scope(name):
            lmcost_hidden_layer = tf.layers.dense(input_tensor, self.config["lmcost_hidden_layer_size"], activation=tf.tanh, kernel_initializer=self.initializer)
            lmcost_output = tf.layers.dense(lmcost_hidden_layer, lmcost_max_vocab_size, activation=None, kernel_initializer=self.initializer)
            lmcost_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=lmcost_output, labels=target_ids)
            lmcost_loss = tf.where(lmcost_mask, lmcost_loss, tf.zeros_like(lmcost_loss))
            return tf.reduce_sum(lmcost_loss)


    def construct_optimizer(self, opt_strategy, loss, learningrate, clip):
        optimizer = None
        if opt_strategy == "adadelta":
            optimizer = tf.train.AdadeltaOptimizer(learning_rate=learningrate)
        elif opt_strategy == "adam":
            optimizer = tf.train.AdamOptimizer(learning_rate=learningrate)
        elif opt_strategy == "sgd":
            optimizer = tf.train.GradientDescentOptimizer(learning_rate=learningrate)
        else:
            raise ValueError("Unknown optimisation strategy: " + str(opt_strategy))

        if clip > 0.0:
            grads, vs = zip(*optimizer.compute_gradients(loss))
            grads, gnorm  = tf.clip_by_global_norm(grads, clip)
            train_op = optimizer.apply_gradients(zip(grads, vs))
        else:
            train_op = optimizer.minimize(loss)
        return train_op


    def preload_word_embeddings(self, embedding_path):
        loaded_embeddings = set()
        embedding_matrix = self.session.run(self.word_embeddings)
        with open(embedding_path, 'r') as f:
            for line in f:
                line_parts = line.strip().split()
                if len(line_parts) <= 2:
                    continue
                w = line_parts[0]
                if self.config["lowercase"] == True:
                    w = w.lower()
                if self.config["replace_digits"] == True:
                    w = re.sub(r'\d', '0', w)
                if w in self.word2id and w not in loaded_embeddings:
                    word_id = self.word2id[w]
                    embedding = numpy.array(line_parts[1:])
                    embedding_matrix[word_id] = embedding
                    loaded_embeddings.add(w)
        self.session.run(self.word_embeddings.assign(embedding_matrix))
        print("n_preloaded_embeddings: " + str(len(loaded_embeddings)))


    def translate2id(self, token, token2id, unk_token, lowercase=False, replace_digits=False, singletons=None, singletons_prob=0.0):
        if lowercase == True:
            token = token.lower()
        if replace_digits == True:
            token = re.sub(r'\d', '0', token)

        token_id = None
        # print(token2id)
        if singletons != None and token in singletons and token in token2id and unk_token != None and numpy.random.uniform() < singletons_prob:
            token_id = token2id[unk_token]
        elif token in token2id:
            token_id = token2id[token]
        elif unk_token != None:
            token_id = token2id[unk_token]
        else:
            raise ValueError("Unable to handle value, no UNK token: " + str(token))
        return token_id


    def create_input_dictionary_for_batch(self, batch, is_training, learningrate):
        sentence_lengths = numpy.array([len(sentence) for sentence in batch])
        max_sentence_length = sentence_lengths.max()
        max_word_length = numpy.array([numpy.array([len(word[0]) for word in sentence]).max() for sentence in batch]).max()
        if self.config["allowed_word_length"] > 0 and self.config["allowed_word_length"] < max_word_length:
            max_word_length = min(max_word_length, self.config["allowed_word_length"])

        word_ids = numpy.zeros((len(batch), max_sentence_length), dtype=numpy.int32)
        char_ids = numpy.zeros((len(batch), max_sentence_length, max_word_length), dtype=numpy.int32)
        word_lengths = numpy.zeros((len(batch), max_sentence_length), dtype=numpy.int32)
        word_labels = numpy.zeros((len(batch), max_sentence_length), dtype=numpy.float32)
        sentence_labels = numpy.zeros((len(batch)), dtype=numpy.float32)
        word_objective_weights = numpy.zeros((len(batch), max_sentence_length), dtype=numpy.float32)
        sentence_objective_weights = numpy.zeros((len(batch)), dtype=numpy.float32)

        singletons = self.singletons if is_training == True else None
        singletons_prob = self.config["singletons_prob"] if is_training == True else 0.0
        
        for i in range(len(batch)):
            count_interesting_labels = numpy.array([1.0 if batch[i][j][-1] != self.config["default_label"] else 0.0 for j in range(len(batch[i]))]).sum()
            sentence_labels[i] = 1.0 if count_interesting_labels >= 1.0 else 0.0

            for j in range(len(batch[i])):
                # print(batch[i][j][0])
                # print(len(self.word2id))
                word_ids[i][j] = self.translate2id(batch[i][j][0], self.word2id, self.UNK, lowercase=self.config["lowercase"], replace_digits=self.config["replace_digits"], singletons=singletons, singletons_prob=singletons_prob)
                word_labels[i][j] = 0.0 if batch[i][j][-1] == self.config["default_label"] else 1.0
                word_lengths[i][j] = len(batch[i][j][0])
                for k in range(min(len(batch[i][j][0]), max_word_length)):
                    char_ids[i][j][k] = self.translate2id(batch[i][j][0][k], self.char2id, self.CUNK)
                
                if len(batch[i][j]) == 2 or (len(batch[i][j]) >= 3 and batch[i][j][1] == "T"):
                    word_objective_weights[i][j] = 1.0
            if len(batch[i][j]) == 2 or (len(batch[i][j]) >= 3 and batch[i][0][1] == "S") or self.config["sentence_objective_persistent"] == True:
                sentence_objective_weights[i] = 1.0

        input_dictionary = {self.word_ids: word_ids, self.char_ids: char_ids, self.sentence_lengths: sentence_lengths, self.word_lengths: word_lengths, self.word_labels: word_labels, self.word_objective_weights: word_objective_weights, self.sentence_labels: sentence_labels, self.sentence_objective_weights: sentence_objective_weights, self.learningrate: learningrate, self.is_training: is_training}
        return input_dictionary

    def process_batch(self, batch, is_training, learningrate):
        feed_dict = self.create_input_dictionary_for_batch(batch, is_training, learningrate)
        cost, sentence_scores, token_scores = self.session.run([self.loss, self.sentence_scores, self.token_scores] + ([self.train_op] if is_training == True else []), feed_dict=feed_dict)[:3]
        return cost, sentence_scores, token_scores


    def initialize_session(self):
        tf.set_random_seed(self.config["random_seed"])
        session_config = tf.ConfigProto()
        session_config.gpu_options.allow_growth = self.config["tf_allow_growth"]
        session_config.gpu_options.per_process_gpu_memory_fraction = self.config["tf_per_process_gpu_memory_fraction"]
        self.session = tf.Session(config=session_config)
        self.session.run(tf.global_variables_initializer())
        self.saver = tf.train.Saver(max_to_keep=1)


    def get_parameter_count(self):
        total_parameters = 0
        for variable in tf.trainable_variables():
            shape = variable.get_shape()
            variable_parameters = 1
            for dim in shape:
                variable_parameters *= dim.value
            total_parameters += variable_parameters
        return total_parameters


    def get_parameter_count_without_word_embeddings(self):
        shape = self.word_embeddings.get_shape()
        variable_parameters = 1
        for dim in shape:
            variable_parameters *= dim.value
        return self.get_parameter_count() - variable_parameters


    def save(self, filename):
        dump = {}
        dump["config"] = self.config
        dump["UNK"] = self.UNK
        dump["CUNK"] = self.CUNK
        dump["word2id"] = self.word2id
        dump["char2id"] = self.char2id
        dump["singletons"] = self.singletons

        dump["params"] = {}
        for variable in tf.global_variables():
            assert(variable.name not in dump["params"]), "Error: variable with this name already exists" + str(variable.name)
            dump["params"][variable.name] = self.session.run(variable)
        with open(filename, 'wb') as f:
            pickle.dump(dump, f, protocol=pickle.HIGHEST_PROTOCOL)


    @staticmethod
    def load(filename, new_config=None):
        with open(filename, 'rb') as f:
            dump = pickle.load(f)

            # for safety, so we don't overwrite old models
            dump["config"]["save"] = None

            # we use the saved config, except for values that are present in the new config
            if new_config != None:
                for key in new_config:
                    dump["config"][key] = new_config[key]

            labeler = MLTModel(dump["config"])
            labeler.UNK = dump["UNK"]
            labeler.CUNK = dump["CUNK"]
            labeler.word2id = dump["word2id"]
            labeler.char2id = dump["char2id"]
            labeler.singletons = dump["singletons"]
            print('len of word2id', len(labeler.word2id))
            labeler.construct_network()
            labeler.initialize_session()

            labeler.load_params(filename)

            return labeler


    def load_params(self, filename):
        with open(filename, 'rb') as f:
            dump = pickle.load(f)

            for variable in tf.global_variables():
                assert(variable.name in dump["params"]), "Variable not in dump: " + str(variable.name)
                assert(variable.shape == dump["params"][variable.name].shape), "Variable shape not as expected: " + str(variable.name) + " " + str(variable.shape) + " " + str(dump["params"][variable.name].shape)
                value = numpy.asarray(dump["params"][variable.name])
                self.session.run(variable.assign(value))

